In [3]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List
from langchain_core.messages import HumanMessage

/Users/yoodongseok/Desktop/rag_project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [32]:
# 상태정의
class RAGState(TypedDict):
    question: str
    docs: List[str]
    summary: str
    answer: str

In [33]:
# Retreive 노드 (문서 검색 노드)
def retrieve_docs(state: RAGState):
    print("Retrieving relevant documents...")
    
    loader = WebBaseLoader("https://ko.wikipedia.org/wiki/대한민국") # loader 생성
    docs = loader.load() # docs에 문서 로드
    
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # splitter 정의
    chunks = splitter.split_documents(docs) # docs 청킹
    
    embeddings = OllamaEmbeddings(model="nomic-embed-text") # 임베딩 모델 정의
    vectordb = Chroma.from_documents(chunks, embedding=embeddings) # 청킹된 문서들을 임베딩시켜서 벡터DB에 저장
    
    retriever = vectordb.as_retriever(search_kwargs={"k":3}) # 유사한 키워드 3개 검색
    retrived_docs = retriever.invoke(state["question"]) # 질문
    
    return {'docs': [d.page_content for d in retrived_docs]}

In [34]:
# Reasoner 노드 (문서 요약 및 핵심 추론)
def reason_over_docs(state: RAGState):
    """검색된 문서를 요약하고 질문에 필요한 핵심 맥락 정의"""
    print("Reasoning about retrieved docs...")
    
    llm = ChatOllama(model="llama3:8b", temperature=0)
    context = "\n\n".join(state["docs"])[:2000]
    
    prompt = f"""
    너는 문서 요약 및 분석 전문가야.
    반드시 한국어로 답변해야 해.
    아래 문서를 바탕으로 질문에 대한 핵심 내용을 정리해줘.
    
    [문서 내용]
    {context}
    
    [질문]
    {state['question']} 
    """
    
    response = llm.invoke(prompt)
    
    return {"summary": response.content}

In [35]:
# Generator 노드 (최종 응답 생성)
def generate_final_answer(state: RAGState):
    """Reasoner가 정리한 내용을 바탕으로 자연어 답변 생성"""
    print("Generating final answer...")
    
    llm = ChatOllama(model="llama3:8b", temperature=0.3)
    
    prompt = f"""
    다음은 사용자의 질문과 관련된 핵심 요약 내용이야.
    이를 바탕으로 반드시 자연스럽고 명확한 한국어로 답변해줘.
    
    [요약된 내용]
    {state["summary"]}
    
    [질문]
    {state["question"]}
    """
    
    response = llm.invoke(prompt)
    return {"answer": response.content}

In [36]:
# Langgraph 구성
workflow = StateGraph(RAGState)
workflow.add_node("retriever", retrieve_docs)
workflow.add_node("reasoner", reason_over_docs)
workflow.add_node("generator", generate_final_answer)

workflow.add_edge(START, "retriever")
workflow.add_edge("retriever", "reasoner")
workflow.add_edge("reasoner", "generator")
workflow.add_edge("generator", END)

app = workflow.compile()

In [37]:
# 실행
if __name__ == "__main__":
    question = "대한민국의 수도는 어디인가요?"
    print(f"Question: {question}")
    
    result = app.invoke({"question": question})
    print("\n✅ 최종 답변:")
    print(result["answer"])

Question: 대한민국의 수도는 어디인가요?
Retrieving relevant documents...
Reasoning about retrieved docs...
Generating final answer...

✅ 최종 답변:
😊

대한민국의 수도는 서울입니다! 🏙️
